# GeoMasterPy Publication Quality Maps

This notebook demonstrates creating publication-quality static maps using Cartopy.

In [ ]:
import ee
import geomasterpy as gmp
import matplotlib.pyplot as plt

# Initialize Earth Engine
ee.Initialize()

## Load Data

In [ ]:
# Define study area (Yellowstone National Park)
yellowstone = ee.Geometry.Rectangle([-111.2, 44.1, -109.9, 45.0])

# Load Landsat 8 data
landsat = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(yellowstone) \
    .filterDate('2023-07-01', '2023-09-30') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .median() \
    .clip(yellowstone)

# Apply scaling factors
def apply_scale_factors(image):
    optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    return image.addBands(optical_bands, None, True)

landsat = apply_scale_factors(landsat)

print("Data loaded successfully")

## Create Publication Map

In [ ]:
# Visualization parameters
vis_params = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
    'gamma': 1.4
}

# Create publication map
fig = gmp.plot_ee_image_cartopy(
    landsat,
    vis_params,
    yellowstone,
    figsize=(12, 8),
    title='Yellowstone National Park - Landsat 8 True Color Composite',
    projection='PlateCarree',
    add_gridlines=True,
    coastlines=False,  # No coastlines for inland area
    borders=True
)

plt.show()

## NDVI Map with Custom Styling

In [ ]:
# Calculate NDVI
ndvi = landsat.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')

# NDVI visualization
ndvi_vis = {
    'bands': ['NDVI'],
    'min': -0.2,
    'max': 0.8,
    'palette': ['brown', 'yellow', 'lightgreen', 'darkgreen']
}

# Create NDVI map
fig = gmp.plot_ee_image_cartopy(
    ndvi,
    ndvi_vis,
    yellowstone,
    figsize=(10, 8),
    title='Yellowstone National Park - Normalized Difference Vegetation Index (NDVI)',
    add_colorbar=True,
    cmap='RdYlGn'  # Override palette with matplotlib colormap
)

plt.show()

## Comparison Map

In [ ]:
# Create before/after comparison (using different seasons)
summer_image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(yellowstone) \
    .filterDate('2023-07-01', '2023-08-31') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .median() \
    .clip(yellowstone)

winter_image = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
    .filterBounds(yellowstone) \
    .filterDate('2023-01-01', '2023-02-28') \
    .filter(ee.Filter.lt('CLOUD_COVER', 10)) \
    .median() \
    .clip(yellowstone)

# Apply scaling
summer_image = apply_scale_factors(summer_image)
winter_image = apply_scale_factors(winter_image)

# Create comparison map
fig = gmp.plotting.create_comparison_map(
    summer_image,
    winter_image,
    vis_params,
    vis_params,
    yellowstone,
    titles=['Summer 2023', 'Winter 2023'],
    figsize=(16, 6)
)

plt.suptitle('Seasonal Comparison - Yellowstone National Park', fontsize=16, fontweight='bold')
plt.show()

## Different Projections

In [ ]:
# Create maps with different projections
projections = ['PlateCarree', 'LambertConformal', 'Mercator']

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, proj in enumerate(projections):
    # Note: This would need to be adapted to work with subplots
    # For now, we'll create individual maps
    pass

print("Different projections can be used:")
for proj in projections:
    print(f"- {proj}")

## Save High-Quality Map

In [ ]:
# Create a high-quality map for publication
fig = gmp.plot_ee_image_cartopy(
    landsat,
    vis_params,
    yellowstone,
    figsize=(12, 10),
    title='Yellowstone National Park\nLandsat 8 True Color Composite (Summer 2023)',
    add_gridlines=True,
    borders=True
)

# Save as high-resolution PNG
gmp.plotting.save_publication_map(
    fig, 
    'yellowstone_publication_map.png', 
    dpi=300, 
    format='png'
)

# Save as PDF for publications
gmp.plotting.save_publication_map(
    fig, 
    'yellowstone_publication_map.pdf', 
    dpi=300, 
    format='pdf'
)

plt.show()

## Custom Styling Tips

In [ ]:
print("Tips for publication-quality maps:")
print("1. Use high DPI (300+) for print publications")
print("2. Choose appropriate projections for your study area")
print("3. Include proper colorbars and legends")
print("4. Use consistent color schemes across figures")
print("5. Add informative titles and captions")
print("6. Consider colorblind-friendly palettes")
print("7. Save in vector formats (PDF, SVG) when possible")